In [1]:
# necessary when running on compute node over NFS
ENV["JULIA_REVISE_POLL"] = "1"
using Revise

In [2]:
using Pkg
pkg"activate .."

  Activating environment at `~/.julia/dev/SymArrays/Project.toml`


In [39]:
using SymArrays
using SymArrays: @cuindex, cudims, SymIndexIter, ind2sub_symgrp, symgrp_sortedsub2ind
using TensorOperations

In [32]:
using CUDA
using MacroTools
using BenchmarkTools

In [28]:
function cuda_contraction_kernel_code(Nsymres)
    # calculate res[iA1,iA3,iS1,iSm2,iS3] = ∑_iA2 A[iA1,iA2,iA3] * S[iS1,iS2,iS3]
    # where iSm2 = (i1,i2,...,iNsymres) and iS2 = sorted(iA2,i1,i2,i3...,iNsymres)
    code = quote
        I = @cuindex(res)
        iA1,iA3,iS1,iSm2,iS3 = I
        ISm2 = ind2sub_symgrp(SI, iSm2)
        res[I...] = zero(eltype(res))
    end
    for n = 0:Nsymres
        iAstart = n==0 ? 1 : :(ISm2[$n]+1)
        iAend = n<Nsymres ? :(ISm2[$(n+1)]) : :(size(A,2))
        iprev = [:( ISm2[$i] ) for i=1:n]
        ipost = [:( ISm2[$i] ) for i=n+1:Nsymres]
        cc = :( for iA2 = $iAstart:$iAend
                    iS2 = symgrp_sortedsub2ind($(iprev...),iA2,$(ipost...))
                    res[I...] += A[iA1,iA2,iA3]*S[iS1,iS2,iS3]
                end)
        push!(code.args,cc)
    end
    push!(code.args,:(return))
    :( @inbounds $code )
end

cuda_contraction_kernel_code (generic function with 1 method)

In [29]:
@generated function SymArrays.cuda_contraction_kernel(res, A, S, SI::SymIndexIter{Nsymres}) where {Nsymres}
    cuda_contraction_kernel_code(Nsymres)
end

In [36]:
?@cuda

```
@cuda [kwargs...] func(args...)
```

High-level interface for executing code on a GPU. The `@cuda` macro should prefix a call, with `func` a callable function or object that should return nothing. It will be compiled to a CUDA function upon first use, and to a certain extent arguments will be converted and managed automatically using `cudaconvert`. Finally, a call to `cudacall` is performed, scheduling a kernel launch on the current CUDA context.

Several keyword arguments are supported that influence the behavior of `@cuda`.

  * `launch`: whether to launch this kernel, defaults to `true`. If `false` the returned kernel object should be launched by calling it and passing arguments again.
  * `dynamic`: use dynamic parallelism to launch device-side kernels, defaults to `false`.
  * arguments that influence kernel compilation: see [`cufunction`](@ref) and [`dynamic_cufunction`](@ref)
  * arguments that influence kernel launch: see [`CUDA.HostKernel`](@ref) and [`CUDA.DeviceKernel`](@ref)


In [30]:
N = 300
A = CUDA.rand(Float64,N)
S = SymArray{(3,),Float64}(CuArray,N,N,N);
S.data .= 1:length(S)
B = CuArray(collect(collect(S)))
C1 = SymArray{(2,),Float64}(CuArray,N,N)
C2 = copy(C1)
@tensor C3[j,k] := A[i]*B[i,j,k]
contract!(C1,A,S,Val(1),Val(1))
# this is the "hand-written" version where A has to be 1D
#contract!(C2,A,S,Val(1))
#@assert C1 ≈ C2
@assert collect(C1) ≈ collect(C3)

In [ ]:
@btime CUDA.@sync @tensor C3[j,k] = A[i]*B[i,j,k]
@btime CUDA.@sync contract!(C1,A,S,Val(1),Val(1));

  953.334 μs (38 allocations: 17.23 KiB)


In [73]:
function SymArrays.contract_symindex!(res::CuArray{TU,5}, A::CuArray{T,3}, ::Val{sizeA13unit}, S::CuArray{U,3}, ::Val{sizeS13unit}, ::Val{Nsym}) where {T,U,TU,sizeA13unit,sizeS13unit,Nsym}
    blk, thr = cudims(res)
    SI = SymIndexIter(Nsym-1,size(A,2))
    #@cuda blocks=blk threads=thr cuda_contraction_kernel(res,A,S,SI)
    kernel = @cuda launch=false cuda_contraction_kernel(res,A,S,SI)
    config = CUDA.launch_configuration(kernel.fun)
    # @show config.threads, thr
    # @show config.blocks, blk
    threads = min(length(res), config.threads)
    blocks = cld(length(res), threads)
    kernel(res,A,S,SI; threads, blocks)
end

In [74]:
contract!(C1,A,S,Val(1),Val(1))

In [75]:
@btime CUDA.@sync contract!(C1,A,S,Val(1),Val(1));

  1.617 ms (28 allocations: 1.62 KiB)
